## Dataset

Polarity Dataset. Pang/Lee ACL 2004

http://www.cs.cornell.edu/people/pabo/movie-review-data/review_polarity.tar.gz

## Data Directories

In [1]:
pos_path = "./review_polarity/txt_sentoken/pos"
neg_path = "./review_polarity/txt_sentoken/neg"

In [2]:
import os

def read_text_files(category_path):
    file_list = os.listdir(category_path)
    texts = []

    for fname in file_list:
        with open(os.path.join(category_path, fname), "r") as f:
            lines = f.readlines()
            texts.extend(lines)
    
    return texts

In [3]:
pos_texts = read_text_files(pos_path)
neg_texts = read_text_files(neg_path)

## Clean

In [4]:
def clean(texts):
    cleaned = []
    for text in texts:
        cleaned_text = text.strip().lower()
        cleaned.append(cleaned_text)

    return cleaned

In [5]:
pos_cleaned = clean(pos_texts)
neg_cleaned = clean(neg_texts)

In [6]:
# merge cleaned
merged = list()
merged.extend(pos_cleaned)
merged.extend(neg_cleaned)

## Build Vocab

In [28]:
from nltk.tokenize import word_tokenize as tokenize
from nltk.corpus import stopwords
from tqdm import tqdm

# ignore stop words
ignore = stopwords.words("english")

# returns a list of unique tokens
def get_words(merged):
    tokens = list()

    for i in tqdm(range(len(merged)), desc="get_words"):
        toks = tokenize(merged[i])
        for t in toks:
            if not t in tokens and t not in ignore:
                tokens.append(t)

    return tokens

In [ ]:
%time words = get_words(merged)

get_words:   2%|▏         | 1374/64720 [00:00<00:42, 1481.99it/s]

In [13]:
len(words)

46319

A lot of words, which means a lot of dims. Need word embeddings to compress this. 

In [24]:
import torchtext
import torch

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## Create dataset with torch

